# Extração e limpeza dos dados de emendas

Neste notebook extraímos e limpamos os dados da relação de emendas por vereador.

Como no notebook anterior, são utilizadas as relações de emendas por vereador para os anos de 2022 a 2025 (ou seja, referentes às LOAs da legislatura eleita em 2020).

In [1]:
import requests
import xml.etree.ElementTree as ET
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from typing import Optional
import os

from config import GRAFICOS_FOLDER, DATA_FOLDER


## Extração de dados

In [2]:
class ExtractEmendasDataYear:

    base_url = 'http://saeows.saopaulo.sp.leg.br/Servico/'

    def __init__(self, service:str, endpoint:str)->None:
        """
        Initialize the ExtractEmendasDataYear class.

        Parameters:
        base_url (str): The base URL for the API.
        endpoint (str): The endpoint for the API.
        """

        if service.startswith('/'):
            service = service[1:]

        if not service.endswith('/'):
            service += '/'

        if not endpoint.startswith('/'):
            endpoint = '/' + endpoint

        if endpoint.endswith('/'):
            endpoint = endpoint[:-1]

        self.base_url = self.base_url + service
        self.endpoint = endpoint

    def __build_url(self, ano: int) -> str:
        """
        Build the URL for the API request.
        """
        return f'{self.base_url}{self.endpoint}?exercicio={ano}'

    def __get_data(self, ano: int) -> str:
        """
        Get data from the API for a specific year.

        Parameters:
        ano (int): The year for which data is to be fetched.

        Returns:
        str: The response text (a XML with JSON data).
        """

        url = self.__build_url(ano)

        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            raise RuntimeError(f"Failed to fetch data for year {ano}: {response.status_code}")

    def __build_xml_tree(self, xml_string:str)-> ET.Element:
        """
        Build an XML tree from a string.

        Parameters:
        xml_string (str): The XML string to be parsed.

        Returns:
        Element: The root element of the parsed XML tree.
        """
        return ET.fromstring(xml_string)

    def __parse_xml_to_json(self, xml_string: str) -> dict:
        """
        Parse the XML string and extract the JSON data.

        Parameters:
        xml_string (str): The XML string to be parsed.

        Returns:
        dict: The extracted JSON data.
        """
        root = self.__build_xml_tree(xml_string)
        json_str = root.text.strip()

        try:
            json_data = json.loads(json_str)
            return json_data

        except json.JSONDecodeError as e:
            raise RuntimeError(f"Erro ao decodificar JSON : {e}")

    def __json_to_dataframe(self, json_data:dict) -> pd.DataFrame:
        """
        Convert JSON data to a pandas DataFrame.

        Parameters:
        json_data (dict): The JSON data to be converted.

        Returns:
        DataFrame: The resulting DataFrame.
        """
        df = pd.json_normalize(json_data)
        return df

    def __pipeline(self, ano:int)->pd.DataFrame:
        """
        Execute the data extraction pipeline for a specific year.

        Parameters:
        ano (int): The year for which data is to be processed.

        Returns:
        DataFrame: The resulting DataFrame.
        """
        xml_string = self.__get_data(ano)
        json_data = self.__parse_xml_to_json(xml_string)
        df = self.__json_to_dataframe(json_data)

        return df

    def __call__(self, ano: int) -> pd.DataFrame:
        """
        Execute the data extraction pipeline for a specific year when the object is called.

        Parameters:
        ano (int): The year for which data is to be processed.

        Returns:
        DataFrame: The resulting DataFrame.
        """

        return self.__pipeline(ano)


In [3]:
ANOS = tuple(range(2022, 2026))


In [4]:
def load_emendas(fname:str, endpoint:str, service:str='IntegracaoPMSP.asmx', anos:[int]=ANOS)->pd.DataFrame:
    """
    Load emendas data for the specified years.

    Parameters:
    anos (list): List of years to load data for.

    Returns:
    DataFrame: The combined DataFrame containing data for all specified years.
    """

    extract_emenda_data = ExtractEmendasDataYear(
        service = service,
        endpoint = endpoint
        )

    fpath = os.path.join(DATA_FOLDER, fname)
    if os.path.exists(fpath):
        print('Loading already downloaded data')
        df = pd.read_csv(fpath, sep=';')
        return df

    dfs = []
    for ano in anos:
        print('Extracting data for year', ano)
        df = extract_emenda_data(ano)
        df['ano'] = ano
        dfs.append(df)

    df = pd.concat(dfs, ignore_index=True)
    df.to_csv(fpath, index=False, sep=';')

    return df

In [5]:
df_emendas_vereadores = load_emendas(fname='emendas_vereaadores.csv',endpoint='EmendasVereadoresJson')

Loading already downloaded data


In [6]:
df_emendas_vereadores['ano'].unique()

array([2022, 2023, 2024, 2025])

In [7]:
df_vereadores = load_emendas(fname='vereadores.csv',endpoint='VereadoresJson')

Loading already downloaded data


In [8]:
df_vereadores['ano'].unique()

array([2022, 2023, 2024, 2025])

In [9]:
df_partidos = load_emendas(fname='partidos.csv', endpoint='PartidosJson')

Loading already downloaded data


In [10]:
df_partidos['ano'].unique()

array([2022, 2023, 2024, 2025])

## Filtrando as colunas

Não vamos usar todas as colunas no estudo. Então agora filtramos os dataframes

In [11]:
df_emendas_vereadores.head()

,Numero,Vereador,DataEmenda,IdExercEmp,Motivo,ano
0,79,1454,02/12/2021,580,Trata-se de importante medida para garantir os...,2022
1,1806,1422,09/12/2021,580,Melhorias diversas.,2022
2,115,1456,07/12/2021,580,Melhoria de Infra estrutura dos bairros dos di...,2022
3,116,1456,07/12/2021,580,Melhoria de infra estrutura dos bairros dos di...,2022
4,117,1456,07/12/2021,580,"No local, ao lado do terreno indicado, existe ...",2022


as emendas de vereadores tem duplicadas. provavelmente por conta da data da proposição. vou ordenar pela data e pegar a ultima

In [12]:
df_emendas_vereadores[['Numero', 'Vereador', 'ano']].duplicated().any()

np.True_

In [13]:
df_emendas_vereadores[['Numero', 'Vereador', 'ano']].duplicated().mean()

np.float64(0.32826187183033656)

In [14]:
df_emendas_vereadores[['Numero', 'Vereador', 'ano', 'Motivo']].duplicated().mean()

np.float64(0.32826187183033656)

In [15]:
df_emendas_vereadores['DataEmenda'] = pd.to_datetime(df_emendas_vereadores['DataEmenda'], format='%d/%m/%Y', errors='coerce')
df_emendas_vereadores.head()

,Numero,Vereador,DataEmenda,IdExercEmp,Motivo,ano
0,79,1454,2021-12-02,580,Trata-se de importante medida para garantir os...,2022
1,1806,1422,2021-12-09,580,Melhorias diversas.,2022
2,115,1456,2021-12-07,580,Melhoria de Infra estrutura dos bairros dos di...,2022
3,116,1456,2021-12-07,580,Melhoria de infra estrutura dos bairros dos di...,2022
4,117,1456,2021-12-07,580,"No local, ao lado do terreno indicado, existe ...",2022


In [16]:
df_emendas_vereadores['DataEmenda'].isna().sum()

np.int64(24)

In [17]:
df_emendas_vereadores = df_emendas_vereadores[df_emendas_vereadores['DataEmenda'].notnull()].reset_index(drop=True)

In [18]:
df_emendas_vereadores = df_emendas_vereadores[df_emendas_vereadores['DataEmenda'].notnull()] \
    .sort_values(by='DataEmenda', ascending=True) \
    .reset_index(drop=True)

Vamos ficar com a ultima propositura de cada emenda

In [19]:
df_emendas_vereadores.drop_duplicates(['Numero', 'Vereador', 'ano'], inplace=True, keep='last')

In [20]:
df_emendas_vereadores = df_emendas_vereadores[['Numero', 'Vereador', 'ano', 'Motivo']].copy()

In [21]:
df_emendas_vereadores.rename(columns={'Numero': 'num_emenda', 'Vereador': 'cod_vereador', 'Motivo' : 'motivo'}, inplace=True)

In [22]:
df_emendas_vereadores.head()

,num_emenda,cod_vereador,ano,motivo
0,79,1454,2022,Trata-se de importante medida para garantir os...
1,85,1451,2022,contratação de equipes de logradouros para at...
2,86,1451,2022,para atendimento de demandas relacionadas a co...
3,87,1451,2022,Atender demanda relacionadas a conservação e m...
4,88,1451,2022,A propositura dos recursos ira levar a comunid...


In [23]:
df_emendas_vereadores.duplicated().any()

np.False_

In [24]:
df_vereadores.head()

,Numero,Apelido,Nome,Partido,ano
0,1423,ANDRÉ SANTOS,ANDRÉ SANTOS,10,2022
1,1430,ATÍLIO FRANCISCO,ATÍLIO FRANCISCO,10,2022
2,1472,Bancada REPUBLICANOS,Bancada REPUBLICANOS,10,2022
3,1473,Liderança REPUBLICANOS,Liderança REPUBLICANOS,10,2022
4,1457,SANSÃO PEREIRA,SANSÃO PEREIRA,10,2022


In [25]:
df_vereadores = df_vereadores[['Numero', 'Apelido', 'Partido', 'ano']].copy()
df_vereadores.rename(columns={'Numero': 'cod_vereador', 'Apelido': 'apelido', 'Partido' : 'cod_partido'}, inplace=True)

In [26]:
df_vereadores.head()

,cod_vereador,apelido,cod_partido,ano
0,1423,ANDRÉ SANTOS,10,2022
1,1430,ATÍLIO FRANCISCO,10,2022
2,1472,Bancada REPUBLICANOS,10,2022
3,1473,Liderança REPUBLICANOS,10,2022
4,1457,SANSÃO PEREIRA,10,2022


In [27]:
df_vereadores.duplicated().any()

np.False_

In [28]:
df_partidos.head()

,Numero,Nome,Sigla,ano
0,10,REPUBLICANOS,REPUBLICANOS,2022
1,11,PARTIDO PROGRESSISTA,PP,2022
2,12,PARTIDO DEMOCRATICO TRABALHISTA,PDT,2022
3,13,PARTIDO DOS TRABALHADORES,PT,2022
4,14,PARTIDO TRABALHISTA BRASILEIRO,PTB,2022


In [29]:
df_partidos = df_partidos[['Numero', 'Sigla', 'Nome', 'ano']].copy()

In [30]:
df_partidos.rename(columns={'Numero': 'cod_partido', 'Sigla': 'sigla', 'Nome' : 'nome_partido'}, inplace=True)

In [31]:
df_partidos.duplicated().any()

np.False_

## Joins


Agora vamos unificar os datasets em um único

### Join vereadores

Unir vereadores e partidos.

In [32]:
df_vereadores['cod_partido'].isnull().any()

np.False_

In [33]:
df_vereadores['cod_partido'].sample(3)  

116    10
144    15
2      10
Name: cod_partido, dtype: int64

In [34]:
df_partidos['cod_partido'].isnull().any()

np.False_

In [35]:
df_partidos['cod_partido'].sample(3)

14     54
53    105
33     13
Name: cod_partido, dtype: int64

In [36]:
df_vereadores_partido = pd.merge(df_vereadores, df_partidos, how='left', on=['ano', 'cod_partido'])

In [37]:
df_vereadores_partido['cod_partido'].isnull().any()

np.False_

In [38]:
df_vereadores_partido.head()

,cod_vereador,apelido,cod_partido,ano,sigla,nome_partido
0,1423,ANDRÉ SANTOS,10,2022,REPUBLICANOS,REPUBLICANOS
1,1430,ATÍLIO FRANCISCO,10,2022,REPUBLICANOS,REPUBLICANOS
2,1472,Bancada REPUBLICANOS,10,2022,REPUBLICANOS,REPUBLICANOS
3,1473,Liderança REPUBLICANOS,10,2022,REPUBLICANOS,REPUBLICANOS
4,1457,SANSÃO PEREIRA,10,2022,REPUBLICANOS,REPUBLICANOS


## Join emendas

In [39]:
df_emendas_vereadores.head()

,num_emenda,cod_vereador,ano,motivo
0,79,1454,2022,Trata-se de importante medida para garantir os...
1,85,1451,2022,contratação de equipes de logradouros para at...
2,86,1451,2022,para atendimento de demandas relacionadas a co...
3,87,1451,2022,Atender demanda relacionadas a conservação e m...
4,88,1451,2022,A propositura dos recursos ira levar a comunid...


In [40]:
df_emendas_vereadores_detalhado = pd.merge(df_emendas_vereadores, df_vereadores_partido, how='left', on=['ano', 'cod_vereador'])

In [41]:
df_emendas_vereadores_detalhado.head()

,num_emenda,cod_vereador,ano,motivo,apelido,cod_partido,sigla,nome_partido
0,79,1454,2022,Trata-se de importante medida para garantir os...,RODRIGO GOULART,41,PSD,PARTIDO SOCIAL DEMOCRATICO
1,85,1451,2022,contratação de equipes de logradouros para at...,RINALDI DIGILIO,107,PSL,PARTIDO SOCIAL LIBERAL
2,86,1451,2022,para atendimento de demandas relacionadas a co...,RINALDI DIGILIO,107,PSL,PARTIDO SOCIAL LIBERAL
3,87,1451,2022,Atender demanda relacionadas a conservação e m...,RINALDI DIGILIO,107,PSL,PARTIDO SOCIAL LIBERAL
4,88,1451,2022,A propositura dos recursos ira levar a comunid...,RINALDI DIGILIO,107,PSL,PARTIDO SOCIAL LIBERAL


In [42]:
df_emendas_vereadores_detalhado['sigla'].isnull().any()

np.False_

In [43]:
df_emendas_vereadores_detalhado.rename({'sigla' : 'sigla_partido', 'motivo' : 'motivo_emenda'}, axis=1, inplace=True)

In [44]:
df_emendas_vereadores_detalhado.groupby('apelido').count()['num_emenda'].nlargest(10)

apelido
SIDNEY CRUZ                  327
DR. SIDNEY CRUZ              150
PROFESSOR TONINHO VESPOLI    134
LUANA ALVES                  129
MILTON LEITE                  57
CELSO GIANNAZI                36
ARSELINO TATTO                34
RICARDO TEIXEIRA              30
SANSÃO PEREIRA                27
ATÍLIO FRANCISCO              25
Name: num_emenda, dtype: int64

In [45]:
df_emendas_vereadores_detalhado.groupby('sigla_partido').count()['num_emenda'].nlargest(10)

sigla_partido
SOLIDARIEDADE    389
PSOL             349
MDB              128
PT               126
UNIÃO            108
PSDB              73
REPUBLICANOS      67
PSD               40
DEM               27
PL                21
Name: num_emenda, dtype: int64

In [46]:
df_emendas_vereadores_detalhado['sigla_partido'].unique()

array(['PSD', 'PSL', 'PT', 'PSB', 'REPUBLICANOS', 'PSDB', 'PL', 'PSOL',
       'MDB', 'PATRIOTA', 'PSC', 'DEM', 'PTB', 'PODE', 'NOVO', 'PV',
       'SOLIDARIEDADE', 'PP', 'UNIÃO', 'AVANTE', 'Sem Filiação'],
      dtype=object)

In [47]:
df_emendas_vereadores_detalhado.to_csv(os.path.join(DATA_FOLDER, 'df_merge_emendas_vereadores.csv'), index=False, sep=';')